##### Dependencias

In [36]:
%load_ext ipybind

The ipybind extension is already loaded. To reload it, use:
  %reload_ext ipybind


In [37]:
from math import sqrt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy

##### Datos

In [28]:
!wget -q -O datos_retocados.csv "https://www.dropbox.com/scl/fi/xnztguety6brdy7t2lfge/wiki_movie_plots_deduped_sample.csv?rlkey=7m867bh7ruilw66qlh7ozl9g4&dl=1"

In [29]:
tabla = pd.read_csv("datos_retocados.csv")

In [30]:
generos_dict = {"western": 0, "science fiction": 1, "romance": 2, "crime": 3}
generos_list = ["western", "science fiction", "romance", "crime"]

In [80]:
train_g, train_v = construir_datos_training(tabla, 1000)

# KNN

##### Python puro

In [38]:
def productoPunto(vectorA, vectorB):
    producto = 0
    for i in range(len(vectorA)) : 
       producto = vectorA[i]*vectorB[i] + producto
    return producto 

def norma(vector):
    valor = 0
    for i in range(len(vector)): 
        valor = vector[i]**2 + valor
    valor = sqrt(valor)
    return valor    

def distanciaCoseno(vectorA, vectorB):
    vecA = vectorA - sum(vectorA) / len(vectorA)
    vecB = vectorB - sum(vectorB) / len(vectorB)
    distancia = 1 - (productoPunto(vecA, vecB) / (norma(vecA) * norma(vecB)))
    # preguntar media
    return distancia

In [41]:
def KVecinos(vector, modelo_generos, modelo_vectores, k):
    vecinos = []
    for i in range(len(modelo_vectores)): 
        vecinos.append((modelo_generos[i], distanciaCoseno(modelo_vectores[i], vector)))
      
    vecinos = sorted(vecinos, key=lambda tup: tup[1])
    kvecinos = []
    for i in range(k):
        kvecinos.append(vecinos[i][0])
    return kvecinos

def KNN(vector, modelo_generos, modelo_vectores, k):
    aparicionesDeClase = [0,0,0,0]

    kvecinos = KVecinos(vector, modelo_generos, modelo_vectores, k)

    for vecino in kvecinos:
        aparicionesDeClase[vecino] = aparicionesDeClase[vecino] + 1
    
    maxIndice = 0
    maxApariciones = 0       
    for i in range(len(aparicionesDeClase)):
        if(aparicionesDeClase[i] > maxApariciones):
            maxIndice = i
            maxApariciones = aparicionesDeClase[i]
    return generos_list[maxIndice]

In [81]:
modelo_generos = train_g[19:]
modelo_vectores = train_v[19:]
for i in range(19):
    vector = train_v[i]
    genero_esperado = generos_list[train_g[i]]
    print("Esperado: " + genero_esperado + " Resultado: " + KNN(vector, modelo_generos, modelo_vectores, 10))

Esperado: crime Resultado: crime
Esperado: romance Resultado: romance
Esperado: science fiction Resultado: science fiction
Esperado: western Resultado: western
Esperado: crime Resultado: science fiction
Esperado: romance Resultado: romance
Esperado: science fiction Resultado: science fiction
Esperado: western Resultado: western
Esperado: crime Resultado: crime
Esperado: romance Resultado: romance
Esperado: science fiction Resultado: science fiction
Esperado: western Resultado: western
Esperado: crime Resultado: crime
Esperado: romance Resultado: romance
Esperado: science fiction Resultado: science fiction
Esperado: western Resultado: western
Esperado: crime Resultado: crime
Esperado: romance Resultado: romance
Esperado: science fiction Resultado: crime


##### Numpy

In [90]:
def normaNumpy(vector):
    return np.linalg.norm(vector)

def distanciaCosenoNumpy(vectorA, vectorB):
    vecA = vectorA - np.average(vectorA)
    vecB = vectorB - np.average(vectorB)
    if (normaNumpy(vecA) == 0 or normaNumpy(vecB) == 0):
        return np.inf # preguntar
    return 1 - (np.dot(vecA, vecB) / (normaNumpy(vecA) * normaNumpy(vecB)))

In [44]:
def KVecinosNumpy(vector, modelo_generos, modelo_vectores, k):
    vecinos = np.zeros((len(modelo_generos), 2))
    vecinos[:, 0] = modelo_generos
    vecinos[:, 1] = [distanciaCosenoNumpy(vector, x) for x in modelo_vectores]
        
    vecinos = vecinos[vecinos[:, 1].argsort()]
    kvecinos = np.array(vecinos[:k, 0], dtype=int)
    return kvecinos


def KNNNumpy(vector, modelo_generos, modelo_vectores, k):
    kvecinos = KVecinosNumpy(vector, modelo_generos, modelo_vectores, k)
    return scipy.stats.mode(kvecinos).mode

In [82]:
modelo_generos = train_g[19:]
modelo_vectores = train_v[19:]
for i in range(19):
    vector = train_v[i]
    genero_esperado = generos_list[train_g[i]]
    print("Esperado: " + genero_esperado + " Resultado: " + generos_list[KNNNumpy(vector, modelo_generos, modelo_vectores, 10)])

Esperado: crime Resultado: crime
Esperado: romance Resultado: romance
Esperado: science fiction Resultado: science fiction
Esperado: western Resultado: western
Esperado: crime Resultado: science fiction
Esperado: romance Resultado: romance
Esperado: science fiction Resultado: romance
Esperado: western Resultado: western
Esperado: crime Resultado: western
Esperado: romance Resultado: romance
Esperado: science fiction Resultado: science fiction
Esperado: western Resultado: western
Esperado: crime Resultado: crime
Esperado: romance Resultado: romance
Esperado: science fiction Resultado: science fiction
Esperado: western Resultado: western
Esperado: crime Resultado: crime
Esperado: romance Resultado: romance
Esperado: science fiction Resultado: crime


# Método de la potencia

##### Implementacion en C++ del metodo de la potencia con deflacion

In [23]:
%%pybind11

#include <iostream>
#include <pybind11/numpy.h>
#include <eigen3/Eigen/Dense>
#include <pybind11/eigen.h>
#include <math.h>
#include <tuple>
#include <vector>
#include <pybind11/stl.h>

using namespace std;
using namespace Eigen;

tuple<double, VectorXd, int> calcular_dominantes(const MatrixXd& matriz, const double tolerancia, const int limite_pasos) {
    VectorXd v_old = VectorXd::Random(matriz.cols());
    v_old = v_old / v_old.lpNorm<2>();
    VectorXd v_new = VectorXd(matriz.cols());
    double diff = 1000000.0;
    int donde_termino = 0;
    for (int i = 0; i < limite_pasos && diff > tolerancia; i++) { 
        v_new = (matriz*v_old) / (matriz*v_old).lpNorm<2>();
        diff = (v_old - v_new).lpNorm<Infinity>();
        v_old = v_new;
        donde_termino++;
    }
    double autovalor = (v_old.transpose() * matriz * v_old)[0] / (v_old.transpose() * v_old)[0];
    return {autovalor, v_old.normalized(), donde_termino};
}

void desinflar(MatrixXd& matriz, const double autoval, VectorXd autovec) {
    matriz = matriz - autoval * autovec * autovec.transpose();
}

vector<tuple<double, VectorXd, int>> metodo_potencia_deflacion(const MatrixXd& matriz, const double tolerancia, const int limite_pasos) {
    MatrixXd mat = matriz;
    vector<tuple<double, VectorXd, int>> res = {};
    for (int i = 0; i < mat.cols(); i++) {
        tuple<double, VectorXd, int> doms;
        //do {          // preguntar que onda el while
        doms = calcular_dominantes(mat, tolerancia, limite_pasos);
        //} while ((mat * get<1>(doms) - get<0>(doms) * get<1>(doms)).lpNorm<Infinity>() > tolerancia);
        res.push_back(doms);

        desinflar(mat, get<0>(doms), get<1>(doms));
    }
    return res;
}

PYBIND11_PLUGIN(metodo_potencia) {
    py::module m("metodo_potencia_con_deflacion");
    m.def("metodo_potencia_deflacion", [](const MatrixXd& matriz, const double tolerancia, const int limite_pasos) {
        return metodo_potencia_deflacion(matriz, tolerancia, limite_pasos);
    });
    m.def("primeros_autos", [](const MatrixXd& matriz, const double tolerancia, const int limite_pasos) {
        return calcular_dominantes(matriz, tolerancia, limite_pasos);
    });
    return m.ptr();
}

##### Prueba de correctitud con Householder

In [24]:
n = 50
for i in range(1, 101): 
    print("Probando matriz " + str(i))
    autovalores_esperados = np.array(range(n, 0, -1), dtype=np.float64) * i

    D = np.diag(autovalores_esperados)
    v = np.ones((n, 1))
    v = v / np.linalg.norm(v)
    H = np.eye(n) - 2 * (v @ v.T)
    M = H @ D @ H.T
    # M es simetrica y tiene autovalores l1 > l2 > ... > l50 > 0
    # Asi que cumple los requisitos
    
    res = metodo_potencia_deflacion(M, 0.000000001, 1000000)

    for j in range(n):
        assert(np.isclose(autovalores_esperados[j], res[j][0]))
        assert(np.allclose(H[j, :], res[j][1]) or np.allclose(H[j, :], res[j][1] * -1)) # Preguntar or
        # Por la forma en la que lo construimos, la columna j de H es el autovector asociado al autovalor lj
        # que al mismo tiempo es el elemento j de la diagonal D. Tambien por lo mismo ya estan normalizados
        # y los que nos devuelve el algoritmo tambien estan normalizados, solo queda tener cuidado con
        # que el sentido sea el opuesto.

print("Todas las matrices dieron bien!")


Probando matriz 1
Probando matriz 2
Probando matriz 3
Probando matriz 4
Probando matriz 5
Probando matriz 6
Probando matriz 7
Probando matriz 8
Probando matriz 9
Probando matriz 10
Probando matriz 11
Probando matriz 12
Probando matriz 13
Probando matriz 14
Probando matriz 15
Probando matriz 16
Probando matriz 17
Probando matriz 18
Probando matriz 19
Probando matriz 20
Probando matriz 21
Probando matriz 22
Probando matriz 23
Probando matriz 24
Probando matriz 25
Probando matriz 26
Probando matriz 27
Probando matriz 28
Probando matriz 29
Probando matriz 30
Probando matriz 31
Probando matriz 32
Probando matriz 33
Probando matriz 34
Probando matriz 35
Probando matriz 36
Probando matriz 37
Probando matriz 38
Probando matriz 39
Probando matriz 40
Probando matriz 41
Probando matriz 42
Probando matriz 43
Probando matriz 44
Probando matriz 45
Probando matriz 46
Probando matriz 47
Probando matriz 48
Probando matriz 49
Probando matriz 50
Probando matriz 51
Probando matriz 52
Probando matriz 53
Pr

##### Error y velocidad de convergencia

In [25]:
# Todos los autovalores de una matriz o solo 1 ?
epsilons = np.logspace(-4, 0, 10)
vectorAdevolver = []
for e in epsilons:

    D = np.diag([10., 10. - e, 5., 2., 1.])
    #v = np.random.rand(5) * 1000 # *1000 para no tener numeros muy chicos
    v = np.ones((5, 1))
    v = v / np.linalg.norm(v)
    H = np.eye(5) - 2 * (v @ v.T)
    M = H @ D @ H.T

    valores = metodo_potencia_deflacion(M, 0.000000001, 1000000)

    # Sacamos el error para cada autovalor y autovector
    errores = [np.linalg.norm(M @ valores[i][1] - valores[i][0] * valores[i][1]) for i in range(len(valores))]
    error_promedio = np.average(errores)
    pasos = [valores[i][2] for i in range(len(valores))]
    print(pasos)
    pasos_promedio = np.average(pasos)
    
    errorYCantidadDePasos = (error_promedio, pasos_promedio)
    vectorAdevolver.append(errorYCantidadDePasos)
#que tiene sentido graficar ya que hay un numero que se hace demasiado grande rompe el promedio jaja saludos recurso
print("")
print(vectorAdevolver)


[720121, 31, 20, 31, 2]
[336227, 32, 22, 29, 2]
[138600, 29, 23, 26, 2]
[52499, 31, 21, 30, 2]
[23052, 30, 21, 34, 2]
[9225, 27, 22, 30, 2]
[2959, 28, 28, 32, 2]
[1366, 32, 24, 28, 2]
[466, 36, 23, 27, 2]
[168, 32, 23, 29, 2]

[(np.float64(1.0362964241851038e-08), np.float64(144041.0)), (np.float64(1.060908871794518e-08), np.float64(67262.4)), (np.float64(1.102723647653988e-08), np.float64(27736.0)), (np.float64(1.0090153872397741e-08), np.float64(10516.6)), (np.float64(1.1051168063741186e-08), np.float64(4627.8)), (np.float64(1.1175395930044718e-08), np.float64(1861.2)), (np.float64(1.0021594825507959e-08), np.float64(609.8)), (np.float64(1.0705687902850247e-08), np.float64(290.4)), (np.float64(9.900800555304429e-09), np.float64(110.8)), (np.float64(1.0682543809099758e-08), np.float64(50.8))]


# PCA

##### Matiz de covarianza

In [76]:
def centrar(vec):
    return vec - np.average(vec)

X = np.apply_along_axis(centrar, axis=1, arr=train_vectores)
X = X.T

matriz_covarianza = (X.T @ X) / (X.shape[1] - 1)

array([[0.77311599, 0.065279  , 0.03921003, ..., 0.06605643, 0.08195611,
        0.0255047 ],
       [0.065279  , 0.34020063, 0.02557367, ..., 0.08988088, 0.01988715,
        0.0593605 ],
       [0.03921003, 0.02557367, 1.03232288, ..., 0.01700313, 0.09670219,
        0.04047649],
       ...,
       [0.06605643, 0.08988088, 0.01700313, ..., 0.31603762, 0.00198119,
        0.03826959],
       [0.08195611, 0.01988715, 0.09670219, ..., 0.00198119, 0.7375047 ,
        0.02324765],
       [0.0255047 , 0.0593605 , 0.04047649, ..., 0.03826959, 0.02324765,
        1.06427586]])

# Clasificacion

##### Construccion de datos

In [77]:
def construir_datos_training(tabla, cantidad_dimensiones):
    tokens = np.hstack(tabla["tokens"].apply(lambda x: x.split()).values)
    unique_tokens = pd.Series(tokens).value_counts().index[:cantidad_dimensiones].values
    unique_tokens_dict = dict(zip(unique_tokens, range(len(unique_tokens))))

    tabla_matriz = np.zeros((len(tabla), len(unique_tokens)), dtype=int)
    for i, row in tabla.iterrows():
        for token in row["tokens"].split():
            if unique_tokens_dict.get(token,False)!=False:
                tabla_matriz[i, unique_tokens_dict[token]] += 1

    train_tabla = tabla.loc[tabla["split"] == "train"].to_numpy()
    train_generos = train_tabla[:, 5]
    train_generos = [generos_dict[x] for x in train_generos]
    train_vectores = np.zeros((len(train_generos), len(unique_tokens)), dtype=int)
    for i in range(len(train_tabla)):
        for token in train_tabla[i, 8].split():
            if unique_tokens_dict.get(token,False) != False:
                train_vectores[i, unique_tokens_dict[token]] += 1

    # Horrible, preguntar como hacer
    tempa = []
    tempb = []
    for i in range(int(len(train_generos)/4)):
        for j in range(4):
            tempa.append(train_generos[i + j*int(len(train_generos)/4)])
            tempb.append(train_vectores[i + j*int(len(train_vectores)/4)])
    train_generos = np.array(tempa)
    train_vectores = np.array(tempb)

    return train_generos, train_vectores

##### Clasificador (renombre de KNN)

In [83]:
def clasificar(vector, modelo_generos, modelo_vectores):
    return KNNNumpy(vector, modelo_generos, modelo_vectores, 5)

##### K-Fold pero sin K ni fold

In [121]:
def performance_clasificador(tabla, cantidad_dimensiones):
    generos, vectores = construir_datos_training(tabla, cantidad_dimensiones)

    correctos = 0
    tamano_particion = int(len(generos) / 5)
    # modelo_generos = generos[tamano_particion:]
    # modelo_vectores = vectores[tamano_particion:]
    modelo_generos = generos[:len(generos) - tamano_particion]
    modelo_vectores = vectores[:len(vectores) - tamano_particion]

    # preguntar si esta bien usar una sola particion siendo que hay tanta diferencia entre cual se elige
    # tarda el quintuple con la ultima particion que con la primera ??????

    # for i in range(tamano_particion):
    for i in range(tamano_particion, len(generos)):
        vector = vectores[i]
        genero_esperado = generos[i]
        genero_res = clasificar(vector, modelo_generos, modelo_vectores)
        if genero_esperado == genero_res:
            correctos = correctos + 1
    return correctos / len(generos)

In [122]:
performances = [performance_clasificador(tabla, q) for q in [500, 1000, 5000]]
performances

[0.60625, 0.628125, 0.63125]

##### K-Fold

In [128]:
def kfold(tabla, cantidad_dimensiones, particiones, vecinos):
    generos, vectores = construir_datos_training(tabla, cantidad_dimensiones)

    correctos = 0
    tamano_particion = int(len(generos) / particiones)
    for i in range(particiones):
        azul_start = i * tamano_particion
        azul_end = i * tamano_particion + tamano_particion
        modelo_generos = np.concatenate((generos[:azul_start], generos[azul_end:]))
        modelo_vectores = np.concatenate((vectores[:azul_start], vectores[azul_end:]))
        for j in range(azul_start, azul_end):
            vector = vectores[j]
            genero_esperado = generos[j]
            genero_res = KNNNumpy(vector, modelo_generos, modelo_vectores, vecinos)
            if genero_esperado == genero_res:
                correctos = correctos + 1
    return correctos / len(generos)

In [137]:
resultados = np.zeros((20, 3))
for k in range(1, 21):
    performances = [kfold(tabla, q, 4, k) for q in [500, 1000, 5000]]
    resultados[k-1, :] = performances
print(resultados)

[[0.66875  0.65625  0.6625  ]
 [0.646875 0.653125 0.65625 ]
 [0.68125  0.7125   0.678125]
 [0.7125   0.728125 0.728125]
 [0.71875  0.721875 0.725   ]
 [0.73125  0.7375   0.740625]
 [0.746875 0.740625 0.75625 ]
 [0.765625 0.734375 0.765625]
 [0.740625 0.746875 0.78125 ]
 [0.75625  0.775    0.7875  ]
 [0.734375 0.7625   0.778125]
 [0.75     0.75     0.784375]
 [0.740625 0.759375 0.7875  ]
 [0.7375   0.753125 0.796875]
 [0.734375 0.7625   0.790625]
 [0.75     0.75     0.790625]
 [0.740625 0.75625  0.79375 ]
 [0.734375 0.734375 0.8     ]
 [0.734375 0.7375   0.8     ]
 [0.746875 0.746875 0.803125]]
